# Web Scraping for Reddit & Predicting Comments

In this project, we will practice two major skills. Collecting data by scraping a website and then building a binary predictor.

As we discussed in week 2, and earlier today, there are two components to starting a data science problem: the problem statement, and acquiring the data.

For this article, your problem statement will be: _What characteristics of a post on Reddit contribute most to the overall interaction (as measured by number of comments)?_

Your method for acquiring the data will be scraping the 'hot' threads as listed on the [Reddit homepage](https://www.reddit.com/). You'll acquire _AT LEAST FOUR_ pieces of information about each thread:
1. The title of the thread
2. The subreddit that the thread corresponds to
3. The length of time it has been up on Reddit
4. The number of comments on the thread

Once you've got the data, you will build a classification model that, using Natural Language Processing and any other relevant features, predicts whether or not a given Reddit post will have above or below the _median_ number of comments.

**BONUS PROBLEMS**
1. If creating a logistic regression, GridSearch Ridge and Lasso for this model and report the best hyperparameter values.
1. Scrape the actual text of the threads using Selenium (you'll learn about this in Webscraping II).
2. Write the actual article that you're pitching and turn it into a blog post that you host on your personal website.

### Scraping Thread Info from Reddit.com

#### Set up a request (using requests) to the URL below. Use BeautifulSoup to parse the page and extract all results

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
URL = "http://www.reddit.com"

In [5]:
## YOUR CODE HERE
r = requests.get(URL)
soup = BeautifulSoup(r.text, 'lxml')

In [6]:
print soup.prettify()

<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   reddit: the front page of the internet
  </title>
  <meta content=" reddit, reddit.com, vote, comment, submit " name="keywords"/>
  <meta content="reddit: the front page of the internet" name="description"/>
  <meta content="always" name="referrer"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="/static/opensearch.xml" rel="search" type="application/opensearchdescription+xml"/>
  <link href="https://www.reddit.com/" rel="canonical"/>
  <meta content="width=1024" name="viewport"/>
  <link href="//out.reddit.com" rel="dns-prefetch"/>
  <link href="//out.reddit.com" rel="preconnect"/>
  <link href="//www.redditstatic.com/desktop2x/img/favicon/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
  <link href="//www.redditstatic.com/desktop2x/img/favicon/apple-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
  <link href="//www.reddits

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The thread title is within an `<a>` tag with the attribute `data-event-action="title"`.
- The time since the thread was created is within a `<time>` tag with attribute `class="live-timestamp"`.
- The subreddit is within an `<a>` tag with the attribute `class="subreddit hover may-blank"`.
- The number of comments is within an `<a>` tag with the attribute data-event-action="comments"`.

## Write 4 functions to extract these items (one function for each): title, time, subreddit, and number of comments.¶
Example
```python
def extract_title_from_result(result):
    return result.find ...
```

##### - Make sure these functions are robust and can handle cases where the data/field may not be available.
>- Remember to check if a field is empty or None for attempting to call methods on it
>- Remember to use try/except if you anticipate errors.

- **Test** the functions on the results above and simple examples

In [31]:
## YOUR CODE HERE
def extract_title(result):
    results = []
    for a in result.find_all('a', attrs={'data-event-action': 'title'}):
        try:
            results.append(a.text)
        except:
            results.append(np.nan)
    return results

def extract_time(result):
    results = []
    for a in result.find_all('time', attrs={'class': 'live-timestamp'}):
        try:
            results.append(a.text)
        except:
            results.append(np.nan)
    return results

def extract_sub(result):
    results = []
    for a in result.find_all('a', attrs={'class': 'subreddit hover may-blank'}):
        try:
            results.append(a.text)
        except:
            results.append(np.nan)
    return results

def extract_comments(result):
    results = []
    for a in result.find_all('a', attrs={'data-event-action': 'comments'}):
        try:
            results.append(a.text)
        except:
            results.append(np.nan)
    return results

In [36]:
import pandas as pd

In [38]:
df1 = pd.DataFrame(zip(extract_title(soup), extract_time(soup), \
                 extract_sub(soup), extract_comments(soup)), \
             columns = ['title', 'time', 'sub', 'comments'])

Now, to scale up our scraping, we need to accumulate more results.

First, look at the source of a Reddit.com page: (https://www.reddit.com/).
Try manually changing the page by clicking the 'next' button on the bottom. Look at how the url changes.

After leaving the Reddit homepage, the URLs should look something like this:
```
https://www.reddit.com/?count=25&after=t3_787ptc
```

The URL here has two query parameters
- count is the result number that the page starts with
- after is the unique id of the last result on the _previous_ page

In order to scrape lots of pages from Reddit, we'll have to change these parameters every time we make a new request so that we're not just scraping the same page over and over again. Incrementing the count by 25 every time will be easy, but the bizarre code after `after` is a bit trickier.

To start off, let's look at a block of HTML from a Reddit page to see how we might solve this problem:
```html
<div class=" thing id-t3_788tye odd gilded link " data-author="LordSneaux" data-author-fullname="t2_j3pty" data-comments-count="1548" data-context="listing" data-domain="v.redd.it" data-fullname="t3_788tye" data-kind="video" data-num-crossposts="0" data-permalink="/r/funny/comments/788tye/not_all_heroes_wear_capes/" data-rank="25" data-score="51468" data-subreddit="funny" data-subreddit-fullname="t5_2qh33" data-timestamp="1508775581000" data-type="link" data-url="https://v.redd.it/ush0rh2tultz" data-whitelist-status="all_ads" id="thing_t3_788tye" onclick="click_thing(this)">
      <p class="parent">
      </p>
      <span class="rank">
       25
      </span>
      <div class="midcol unvoted">
       <div aria-label="upvote" class="arrow up login-required access-required" data-event-action="upvote" role="button" tabindex="0">
       </div>
       <div class="score dislikes" title="53288">
        53.3k
       </div>
       <div class="score unvoted" title="53289">
        53.3k
       </div>
       <div class="score likes" title="53290">
        53.3k
       </div>
       <div aria-label="downvote" class="arrow down login-required access-required" data-event-action="downvote" role="button" tabindex="0">
       </div>
      </div>
```

Notice that within the `div` tag there is an attribute called `id` and it is set to `"thing_t3_788tye"`. By finding the last ID on your scraped page, you can tell your _next_ request where to start (pass everything after "thing_").

For more info on this, you can take a look at the [Reddit API docs](https://github.com/reddit/reddit/wiki/JSON)

## Write one more function that finds the last `id` on the page, and stores it.

In [40]:
import re

In [45]:
## YOUR CODE HERE
def extract_code(result):
    return result.find(id=re.compile("thing"))['id'].replace("thing_","")

In [48]:
code = extract_code(soup)

## (Optional) Collect more information

While we only require you to collect four features, there may be other info that you can find on the results page that might be useful. Feel free to write more functions so that you have more interesting and useful data.

In [ ]:
## YOUR CODE HERE

## Now, let's put it all together.

Use the functions you wrote above to parse out the 4 fields - title, time, subreddit, and number of comments. Create a dataframe from the results with those 4 columns.

In [67]:
from selenium import webdriver

In [81]:
driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver")
driver.get("http://www.reddit.com/")
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
driver.close()

In [47]:
import time
print 'hi'
time.sleep(5)
print 'there'

hi
there


In [83]:

code = extract_code(soup)
print code

t3_7k35nv


In [84]:
url_template = "http://www.reddit.com/?count={}&after={}"
max_results = 5000 # Set this to a high-value (5000) to generate more results. 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.

data = pd.DataFrame()

for start in range(25, max_results, 25):
    url = url_template.format(start, code)
    print url
    driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver")
    driver.get("http://www.reddit.com/")
    time.sleep(3)
    html = driver.page_source
    driver.close()
    soup = BeautifulSoup(html, 'lxml')
    code = extract_code(soup)
    print code
    temp = pd.DataFrame(zip(extract_title(soup), extract_time(soup), \
                 extract_sub(soup), extract_comments(soup)), \
             columns = ['title', 'time', 'sub', 'comments'])
    data = pd.concat([data, temp])
    time.sleep(5)
    
    pass

http://www.reddit.com/?count=25&after=t3_7k35nv
t3_7g6qi3
http://www.reddit.com/?count=50&after=t3_7g6qi3
t3_7m4404
http://www.reddit.com/?count=75&after=t3_7m4404
t3_7k35nv
http://www.reddit.com/?count=100&after=t3_7k35nv
t3_7k35au
http://www.reddit.com/?count=125&after=t3_7k35au
t3_7k35nv
http://www.reddit.com/?count=150&after=t3_7k35nv
t3_7m4404
http://www.reddit.com/?count=175&after=t3_7m4404
t3_7m4404
http://www.reddit.com/?count=200&after=t3_7m4404
t3_7m4404
http://www.reddit.com/?count=225&after=t3_7m4404
t3_7m4404
http://www.reddit.com/?count=250&after=t3_7m4404
t3_7m4404
http://www.reddit.com/?count=275&after=t3_7m4404
t3_7m4404
http://www.reddit.com/?count=300&after=t3_7m4404
t3_7m4404
http://www.reddit.com/?count=325&after=t3_7m4404
t3_7m4404
http://www.reddit.com/?count=350&after=t3_7m4404
t3_7m4404
http://www.reddit.com/?count=375&after=t3_7m4404
t3_7m4404
http://www.reddit.com/?count=400&after=t3_7m4404
t3_7m4404
http://www.reddit.com/?count=425&after=t3_7m4404
t3_7ii8o2


t3_7m4404
http://www.reddit.com/?count=3475&after=t3_7m4404
t3_7mhuuf
http://www.reddit.com/?count=3500&after=t3_7mhuuf
t3_7m4404
http://www.reddit.com/?count=3525&after=t3_7m4404
t3_7m4404
http://www.reddit.com/?count=3550&after=t3_7m4404
t3_7k35yw
http://www.reddit.com/?count=3575&after=t3_7k35yw
t3_7m4404
http://www.reddit.com/?count=3600&after=t3_7m4404
t3_7ebni5
http://www.reddit.com/?count=3625&after=t3_7ebni5
t3_7k35nv
http://www.reddit.com/?count=3650&after=t3_7k35nv
t3_767ifg
http://www.reddit.com/?count=3675&after=t3_767ifg
t3_7m4404
http://www.reddit.com/?count=3700&after=t3_7m4404
t3_7ebp6d
http://www.reddit.com/?count=3725&after=t3_7ebp6d
t3_7ebp6d
http://www.reddit.com/?count=3750&after=t3_7ebp6d
t3_7k35nv
http://www.reddit.com/?count=3775&after=t3_7k35nv
t3_7m4404
http://www.reddit.com/?count=3800&after=t3_7m4404
t3_7ii8o2
http://www.reddit.com/?count=3825&after=t3_7ii8o2
t3_767ifg
http://www.reddit.com/?count=3850&after=t3_767ifg
t3_7m4404
http://www.reddit.com/?count=3

In [87]:
data.to_csv('top.csv', index=False, encoding='utf-8')

In [ ]:
## YOUR CODE HERE

### Save your results as a CSV
You may do this regularly while scraping data as well, so that if your scraper stops of your computer crashes, you don't lose all your data.

In [ ]:
# Export to csv


## Predicting comments using Random Forests + Another Classifier

#### Load in the the data of scraped results

In [ ]:
## YOUR CODE HERE

#### We want to predict a binary variable - whether the number of comments was low or high. Compute the median number of comments and create a new binary variable that is true when the number of comments is high (above the median)

We could also perform Linear Regression (or any regression) to predict the number of comments here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW number of comments.

While performing regression may be better, performing classification may help remove some of the noise of the extremely popular threads. We don't _have_ to choose the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of comment numbers. 

In [ ]:
## YOUR CODE HERE

#### Thought experiment: What is the baseline accuracy for this model?

In [ ]:
## YOUR CODE HERE

#### Create a Random Forest model to predict High/Low number of comments using Sklearn. Start by ONLY using the subreddit as a feature. 

In [ ]:
## YOUR CODE HERE

#### Create a few new variables in your dataframe to represent interesting features of a thread title.
- For example, create a feature that represents whether 'cat' is in the title or whether 'funny' is in the title. 
- Then build a new Random Forest with these features. Do they add any value?
- After creating these variables, use count-vectorizer to create features based on the words in the thread titles.
- Build a new random forest model with subreddit and these new features included.

In [ ]:
## YOUR CODE HERE

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy of the model, as well as any other metrics you feel are appropriate. 

In [ ]:
## YOUR CODE HERE

#### Repeat the model-building process with a non-tree-based method.

In [ ]:
## YOUR CODE HERE

#### Use Count Vectorizer from scikit-learn to create features from the thread titles. 
- Examine using count or binary features in the model
- Re-evaluate your models using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [ ]:
## YOUR CODE HERE

# Executive Summary
---
Put your executive summary in a Markdown cell below.

### BONUS
Refer to the README for the bonus parts

In [ ]:
## YOUR CODE HERE